# Exploring ``np.linalg.solve``

Kai Striega

In [1]:
import numpy as np
import scipy as sp
from numpy.testing import assert_allclose
np.__version__, sp.__version__

('1.26.4', '1.13.0')

# Problem Statement

Consider the linear system of equations:

$$ Ax = b $$

Where: 

* $A$ is a known $n\text{ x }n$ matrix
* $x$ is an unknown $n\text{ x }1$ matrix
* $b$ is a known $n\text{ x }1$ matrix

We also assume that $A$ is __invertible__, that is, there exists another $n\text{ x }n$ matrix, $B$, such that $BA = I_n$.

We want to be able to solve this system for $x$.

* We've all solved these kind of problems by hand
* In applications the problem size is usually huge
* In this case we need to develop algorithms to let a computer do the job
* Well, we don't, because they have already been written

In [2]:
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]])

b = np.array([8, -11, -3])
x = np.linalg.solve(A, b)

A @ x

array([  8., -11.,  -3.])

In [3]:
rng = np.random.default_rng(42)
n = 10_000
A = rng.random((n, n))
b = rng.random(n)
x = np.linalg.solve(A, b)
assert_allclose(A @ x, b)

## Great!

* We have reduced a whole subfield of mathematics into a single line of NumPy.
* But I like to __understand__ how a method works, not just use it
* This talk will look at __how__ to solve this type of equation

#  A naive method: matrix inversion

* From our maths classes, we know that we can invert a matrix to solve a system of equations
* This can be used to solve a system of equations:
$$
\begin{align}
Ax &= b \\
A^{-1}Ax &= A^{-1}b \\
x &= A^{-1}b \\
\end{align}
$$

In [4]:
x = np.linalg.inv(A) @ b
assert_allclose(A @ x, b)

In [5]:
%timeit np.linalg.inv(A) @ b

6.87 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit np.linalg.solve(A, b)

2.24 s ± 71.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# How does NumPy do it?

* NumPy defines the ``solve`` function in [umath_linalg.cpp](https://github.com/numpy/numpy/blob/main/numpy/linalg/umath_linalg.cpp#L1728-L1765)
* NumPy calls the [LAPACK](https://en.wikipedia.org/wiki/LAPACK) routine ``_dgesv``
* The reference implementation of ``_dgesv`` is available on [Netlib](https://netlib.org/lapack/explore-html-3.6.1/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html)

# LU Factorization

* Basic idea: Convert $A$ to an upper triangular matrix by multiplying by finitely many lower triangular matrices

$$ E_{n-1} \dots  E_2 E_1 A = U $$

* With some rearranging, this leads to the LU factorisation:
  
$$ 
\begin{align}
A &= E_1^{-1}E_2^{-1} \ldots E_{n-1}^{-1}U \\
&= L_1 L_2 \ldots L_{n-1} U \\
&= LU
\end{align}
$$

## Solving linear systems using LU Factorization

* Consider the linear system $$Ax = b$$ with LU factorisation $$ A = LU $$. 
* Now we have $LUx = b$. Set $y = Ux$ Then the solution for $Ax=b$ can be found in two steps:
    1. Solve the system $Ly=b$ for $y$ by _forward substitution_
    2. Solve the system $Ux=y$ for $x$ by _back substitution_

### Forward substitution

Consider the system $Ly=b$:

$$ Ly = 
\begin{bmatrix}
l_{1, 1} & 0 & \ldots & 0 \\ 
l_{2, 1} & l_{2, 2} & & \vdots\\
\vdots & & \ddots & 0 \\
l_{n, 1} & l_{n, 2} & \ldots & l_{n,n}
\end{bmatrix}
\begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}
= 
\begin{bmatrix} b_1 \\ b_2 \\ \vdots \\ b_n \end{bmatrix}
$$

By the first equation we get $$y_1 = \frac{b_1}{l_{1,1}}$$ 

Then, by induction, it can be shown that the solution is given by forward substitution:

$$ y_i = \frac{b_i - \sum_{k=1}^{i-1} l_{i,k}y_k}{l_{i,i}} $$

### Back Substituion

* We now know $y$ we need to solve the system $Ux=y$ for $x$ by back substitution:

$$ U = 
\begin{bmatrix}
u_{1, 1} & u_{1,2} & \ldots & u_{1,n} \\ 
0 & l_{2, 2} & \ldots & u_{2,n}\\
\vdots & & \ddots & \vdots \\
0 & \ldots & 0 & u_{n,n}
\end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{bmatrix}
= 
\begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}
$$

* By the last equation we obtain: $$ x_n = \frac{y_n}{u_{n,n}} $$ 
* By induction we can solve the solution $x$ by back substiution:

$$ x_i = \frac{y_i - \sum_{k=i + 1}^{n} u_{i,k}x_k}{u_{i,i}} $$

#  All that in code

In [7]:
def lu(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    U = A.copy().astype(np.float64)
    L = np.eye(n, dtype=np.float64)

    for i in range(n):
            
        # Eliminate entries below i with row operations 
        # on U and reverse the row operations to 
        # manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U

In [8]:
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]])
b = np.array([8, -11, -3])

L, U = lu(A)

In [9]:
L

array([[ 1. ,  0. ,  0. ],
       [-1.5,  1. ,  0. ],
       [-1. ,  4. ,  1. ]])

In [10]:
U

array([[ 2. ,  1. , -1. ],
       [ 0. ,  0.5,  0.5],
       [ 0. ,  0. , -1. ]])

In [11]:
L @ U

array([[ 2.,  1., -1.],
       [-3., -1.,  2.],
       [-2.,  1.,  2.]])

In [12]:
assert_allclose(L @ U, A)

In [13]:
def forward_substitution(L, b):
    
    # Get number of rows
    n = L.shape[0]
    
    # Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    # Here we perform the forward-substitution.  
    # Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    # Looping over rows in reverse (from the bottom  up),
    # starting with the second to last row, because  the 
    # last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

In [14]:
y = forward_substitution(L, b)
y

array([8., 1., 1.])

In [15]:
def back_substitution(U, y):
    
    # Number of rows
    n = U.shape[0]
    
    # Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    # Here we perform the back-substitution.  
    # Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    # Looping over rows in reverse (from the bottom up), 
    # starting with the second to last row, because the 
    # last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

In [16]:
x = back_substitution(U, y)
x

array([ 2.,  3., -1.])

In [17]:
assert_allclose(A @ x, b)

# Problems with LU factorisation

## Problem 1: Backward instability

Consider the linear system $Ax=b$, where:

$$
A = \begin{bmatrix} 1 & 1 & 1 \\ 2 & 2 + \varepsilon & 5 \\ 4 & 6 & 8 \\ \end{bmatrix}, b = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}
$$

Then 

$$
E_1 A 
= \begin{bmatrix} 1 & 0 & 0 \\ -2 & 1 & 0 \\ -4 & 0 & 0 \\ \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\ 2 & 2 + \varepsilon & 5 \\ 4 & 6 & 8 \\ \end{bmatrix} 
= \begin{bmatrix} 1 & 1 & 1 \\ 0 & \varepsilon & 3 \\ 0 & 2 & 4 \\ \end{bmatrix}
$$

And 

$$ E_2 E_1 A
= \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & -\frac{2}{\varepsilon} & 1 \\ \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\ 0 & \varepsilon & 3 \\ 0 & 2 & 4 \\ \end{bmatrix}
= \begin{bmatrix} 1 & 1 & 1 \\ 0 & \varepsilon & 3 \\ 0 & 0 & 4-\frac{6}{\varepsilon} \\ \end{bmatrix}
$$

Which gives:

$$
L = \begin{bmatrix} 1 & 0 & 0 \\ 2 & 1 & 0 \\ 4 & \frac{2}{\varepsilon} & 1 \end{bmatrix}, U = \begin{bmatrix} 1 & 1 & 1 \\ 0 & \varepsilon & 3 \\ 0 & 0 & 4-\frac{6}{\varepsilon} \\ \end{bmatrix}
$$

* If $\varepsilon$ is on the order of machine accuracy, the $4$ in $4-\frac{6}{\varepsilon}$ becomes insignifcant (because $\frac{6}{\varepsilon}$ is very large)
* This means that during computation we actually use:

$$ L^{\prime} = L, U^{\prime} = \begin{bmatrix} 1 & 1 & 1 \\ 0 & \varepsilon & 3 \\ 0 & 0 & -\frac{6}{\varepsilon} \\ \end{bmatrix} $$

But now:

$$ 
L^{\prime} U^{\prime}
= \begin{bmatrix} 1 & 1 & 1 \\ 2 & 2 + \varepsilon & 5 \\ 4 & 6 & 4 \\ \end{bmatrix}
\ne A
$$

... which is a problem

Furthermore if we use $L$ and $U$ to get $x$:

$$
x = \begin{bmatrix} \frac{4 \varepsilon - 7}{2 \varepsilon - 3} \\ \frac{2}{2\varepsilon - 3} \\ -\frac{2(\varepsilon - 1)}{2 \varepsilon - 3} \end{bmatrix}
\approx \begin{bmatrix} \frac{7}{3} \\ -\frac{2}{3} \\ -\frac{2}{3} \\ \end{bmatrix}
$$

While if we use $L^{\prime}$ and $U^{\prime}$:
$$
x^{\prime}
= \begin{bmatrix} \frac{11 - 2 \varepsilon}{3} \\ -2 \\ \frac{2\varepsilon - 2}{3} \end{bmatrix}
\approx \begin{bmatrix} \frac{11}{3} \\ -2 \\ -\frac{2}{3} \end{bmatrix}
$$

Clearly $x$ and $x^{\prime}$ are signifficantly different even when $L^{\prime}$ and $U^{\prime}$ are close to $L$ and $U$

## Problem 2: Not all matrices have an LU factorisation

Consider the matrix:

$$ A = \begin{bmatrix} 1 & 1 & 1 \\ 2 & 2 & 5 \\ 4 & 6 & 8 \\ \end{bmatrix} $$

Then:

$$ 
E_1 A 
= \begin{bmatrix} 1 & 0 & 0 \\ -2 & 1 & 0 \\ -4 & 0 & 1 \\ \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\ 2 & 2 & 5 \\ 4 & 6 & 8 \\ \end{bmatrix}
= \begin{bmatrix} 1 & 1 & 1 \\ 0 & 0 & 3 \\ 0 & 2 & 4 \\ \end{bmatrix}
$$

* The standard algorithm cannot go any further (since we would have to divide by zero)
* This implies that $A$ does not have an LU Factorisation

In [18]:
A = np.array([[1, 1, 1],
              [2, 2, 5],
              [4, 6, 8]])
lu(A)

/tmp/ipykernel_21838/2020444748.py:14: RuntimeWarning: divide by zero encountered in divide
  factor = U[i+1:, i] / U[i, i]
/tmp/ipykernel_21838/2020444748.py:16: RuntimeWarning: invalid value encountered in multiply
  U[i+1:] -= factor[:, np.newaxis] * U[i]


(array([[ 1.,  0.,  0.],
        [ 2.,  1.,  0.],
        [ 4., inf,  1.]]),
 array([[  1.,   1.,   1.],
        [  0.,   0.,   3.],
        [ nan,  nan, -inf]]))

## Pivoting

* Since the order of our equations does not matter we can swap rows
* We do this using a [permuation matrix](https://mathworld.wolfram.com/PermutationMatrix.html), $P_i$
* Each permutation matrix allows us to swap different rows

* Let's look at coloumn 1 of $A$
* We can see the number of largest magnitude is 4
* Let's swap rows 1 and 3

$$ P_1 A
= \begin{bmatrix} 0 & 0 & 1 \\ 0 & 1 & 0 \\ 1 & 0 & 0 \\ \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\ 2 & 2 & 5 \\ 4 & 6 & 8 \\ \end{bmatrix} 
= \begin{bmatrix} 4 & 6 & 8 \\ 2 & 2 & 5 \\ 1 & 1 & 1 \\ \end{bmatrix} 
$$

* We can now perform the first step of the algorithm

$$ 
E_1 P_1 A
= \begin{bmatrix} 1 & 0 & 0 \\ -\frac{1}{2} & 1 & 0 \\ -\frac{1}{4} & 0 & 1 \\ \end{bmatrix} \begin{bmatrix} 4 & 6 & 8 \\ 2 & 2 & 5 \\ 1 & 1 & 1 \\ \end{bmatrix}
= \begin{bmatrix} 4 & 6 & 8 \\ 0 & -1 & 1 \\ 0 & -\frac{1}{2} & -1 \\ \end{bmatrix}
$$

* We can now continue with our algorithm, pivoting as required

* In general, back instability can be avoided by swapping rows before applying $E_k$ i.e. we perform:

$$
E_{n-1}P_{n-1} \ldots E_2P_2E_1P_1A = U
$$

* With some rejiggling, this can be transformed to show that

$$ PA = LU $$

* i.e. we have some permutation matrix $P$ that, when multiplied with $A$, gives us the LU factorisation

# Can we do better?

* LU Factorisation is good for dense, small systems
* However, in reality many problems are  sparse, large or have special structure
* [SciPy](https://scipy.org/) contains solvers for sparse systems or systems with special structure

## scipy.linalg

* Inlcudes paths for dense matrices of special structure
    * ``solve``: symmetric, hermitian and positive definite matrices
    * ``solve_banded``
    * ``solveh_banded``
    * ``solve_circulant``
    * ``solve_triangular``
    * ``solve_toeplitz``

## scipy.sparse.linalg.spsolve

* Takes an ndarray or sparse array and solves it
* Will conver ndarrays to a CSC or CSR array
* This might add some overhead
* Recommend to use sparse arrays when working with sparse systems

In [19]:
n = 10_000
rng = np.random.default_rng(42)
A = rng.random((n, n))
b = rng.random(n)

In [20]:
# Create a sparse matrix by zeroing 99% of rows
row_ind = rng.choice(np.arange(n), int(n * 99 / 100), replace=False)
A[row_ind, :] = 0
np.fill_diagonal(A, 1)  # Make sure a valid solution exists

In [21]:
%timeit np.linalg.solve(A, b)

1.95 s ± 38.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit sp.sparse.linalg.spsolve(A, b)

<magic-timeit>:1: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format


392 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
R = sp.sparse.csr_array(A)
%timeit sp.sparse.linalg.spsolve(R, b)

18.6 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
